In [7]:
# --- Import dependencies ---
import pandas as pd

In [8]:
# --- Define input and output file paths ---
FIREWOOD_FILE = "../data/processed/fuel_consumption_master_dataset_2010_2024.csv"
CLEANFUEL_FILE = "../data/processed/cameroon_clean_cooking_simple_2010_2024.csv"
OUTFILE = "../data/raw/merged_raw_hybrid.csv"

In [9]:
# --- STEP 3: Load both datasets ---
fw = pd.read_csv(FIREWOOD_FILE)
cf = pd.read_csv(CLEANFUEL_FILE)

print("Firewood dataset shape:", fw.shape)
print("Clean fuel dataset shape:", cf.shape)

Firewood dataset shape: (5479, 25)
Clean fuel dataset shape: (5479, 12)


In [10]:
# --- STEP 4: Normalize column names ---
fw.columns = fw.columns.str.lower()
cf.columns = cf.columns.str.lower()

In [11]:
# --- STEP 4: Identify region/year columns in firewood dataset ---
def find_col(df, candidates):
    for c in candidates:
        for col in df.columns:
            if c in col:
                return col
    return None

region_fw = find_col(fw, ['region','province','admin_name','adm_name','shprov'])
year_fw = find_col(fw, ['year','date','survey_year'])

if region_fw is None or year_fw is None:
    raise KeyError(
        f"Could not find region or year column in firewood dataset.\n"
        f"Available columns:\n{fw.columns.tolist()}"
    )

fw = fw.rename(columns={region_fw:'region', year_fw:'year'})
print(f"\n✅ Using columns: region='{region_fw}', year='{year_fw}'")


KeyError: "Could not find region or year column in firewood dataset.\nAvailable columns:\n['date', 'temperature_avg', 'rainfall_mm', 'humidity_avg', 'wind_speed_kmh', 'season', 'woodfuel_production_m3', 'year', 'month', 'quarter', 'day_of_year', 'day_of_week', 'is_weekend', 'temp_7d_avg', 'temp_30d_avg', 'temperature_lag1', 'rainfall_7d_avg', 'rainfall_30d_cumulative', 'rainfall_lag1', 'woodfuel_7d_avg', 'woodfuel_30d_avg', 'woodfuel_lag1', 'woodfuel_lag7', 'is_dry_season', 'is_rainy_season']"

In [6]:
# --- STEP 5: Clean region/year columns ---
fw['region'] = fw['region'].astype(str).str.strip().str.lower()
fw['year'] = pd.to_numeric(fw['year'], errors='coerce').astype('Int64')


KeyError: 'region'

In [22]:
# --- STEP 6: Aggregate firewood dataset to unique region-year ---
fw_agg = fw.groupby(['region','year'], as_index=False).mean(numeric_only=True)
print(f"✅ Aggregated firewood dataset shape: {fw_agg.shape}")
print(f"Unique region-year pairs: {fw_agg[['region','year']].drop_duplicates().shape[0]}")



✅ Hybrid merge complete!
Merged shape: (2001299, 36)


,date_x,temperature_avg,rainfall_mm,humidity_avg,wind_speed_kmh,season,woodfuel_production_m3,year,month_x,quarter_x,...,month_y,quarter_y,day_of_year_y,day_of_week_y,is_weekend_y,clean_fuel_access,country,country_code,series_name,series_code
0,2010-01-01,12.0,0.2,73.0,10.1,Dry,37258.41,2010,1,1,...,1,1,1,4,0,2.000,Cameroon,CMR,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS
1,2010-01-01,12.0,0.2,73.0,10.1,Dry,37258.41,2010,1,1,...,1,1,2,5,1,2.000,Cameroon,CMR,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS
2,2010-01-01,12.0,0.2,73.0,10.1,Dry,37258.41,2010,1,1,...,1,1,3,6,1,2.001,Cameroon,CMR,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS
3,2010-01-01,12.0,0.2,73.0,10.1,Dry,37258.41,2010,1,1,...,1,1,4,0,0,2.001,Cameroon,CMR,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS
4,2010-01-01,12.0,0.2,73.0,10.1,Dry,37258.41,2010,1,1,...,1,1,5,1,0,2.005,Cameroon,CMR,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS


In [24]:
# --- STEP 7: Prepare clean-fuel dataset (national data only) ---
year_cf = find_col(cf, ['year','date'])
cf = cf.rename(columns={year_cf:'year'})
cf['year'] = pd.to_numeric(cf['year'], errors='coerce').astype('Int64')

# Keep only year + numeric clean-fuel columns
cf_numeric = cf.select_dtypes(include='number').copy()
cf_numeric['year'] = cf['year']
cf_numeric = cf_numeric.drop_duplicates(subset=['year'])

print(f"✅ Clean-fuel dataset (national, yearly) shape: {cf_numeric.shape}")
display(cf_numeric.head())



💾 Saved hybrid merged dataset as ../data/raw/merged_raw_hybrid.csv


In [ ]:
# --- STEP 8: Merge clean-fuel data by YEAR only ---
merged = pd.merge(fw_agg, cf_numeric, on='year', how='left')

In [ ]:
# --- STEP 9: Sanity checks ---
dupes = merged.duplicated(subset=['region','year']).sum()
print(f"\n✅ Merge complete! Final dataset shape: {merged.shape}")
print(f"Duplicate region-year rows: {dupes}")

In [ ]:
# --- STEP 10: Save the clean hybrid dataset ---
merged.to_csv(OUTFILE, index=False)
print(f"\n💾 Saved clean hybrid dataset: {OUTFILE}")

In [ ]:
# --- STEP 11: Preview result ---
display(merged.head(10))